# POPULARITY BASED SYSTEM

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout
from popular_movie import  recommend_movies
from IPython.core.display import HTML


movies_data = pd.read_csv('movies.csv')
ratings_data = pd.read_csv('ratings.csv')

movie_ratings = pd.merge(movies_data, ratings_data, on='movieId')        


# genre_input = widgets.Text(description = 'Genre (g) : ')
# min_reviews_threshold_input = widgets.IntText(description = 'Minimum Reviews (t) : ')
# num_recommendations_input = widgets.IntText(description = 'Number of Recommendations (N) : ')

description_genre = widgets.Label('Genre (g):')
genre_input = widgets.Text(value='')
genre_hbox = widgets.HBox([description_genre , widgets.Label('  '), genre_input])

description_reviews = widgets.Label('Minimum Reviews (t):')
review_input = widgets.IntText(value=0)
review_hbox = widgets.HBox([description_reviews , widgets.Label('  '), review_input])


description_recommendations = widgets.Label('Number of Recommendations (N):')
recommendation_input = widgets.IntText(value=0)
recommendation_hbox = widgets.HBox([description_recommendations , widgets.Label('  '), recommendation_input])


recommend_button = widgets.Button(description = 'Recommend Movies')
output = widgets.Output()

def recommend_movies_on_button_click(b):
        with output:
            output.clear_output()
            genre = genre_input.value
            min_reviews_threshold = review_input.value
            num_recommendations = recommendation_input.value

            top_recommendations = recommend_movies(movies_data,ratings_data, genre, min_reviews_threshold, num_recommendations)

            print(f"Top {num_recommendations} Recommended Movies in the '{genre}' Genre (Minimum {min_reviews_threshold} Reviews):\n")
            print(top_recommendations[['title', 'rating', 'Reviews Count']])

# Link the button click event to the function
recommend_button.on_click(recommend_movies_on_button_click)

# Display widgets
display(genre_hbox, review_hbox, recommendation_hbox, recommend_button, output)


       movieId                                title  \
0            1                     Toy Story (1995)   
1            2                       Jumanji (1995)   
2            3              Grumpier Old Men (1995)   
3            4             Waiting to Exhale (1995)   
4            5   Father of the Bride Part II (1995)   
...        ...                                  ...   
10324   146684        Cosmic Scrat-tastrophe (2015)   
10325   146878           Le Grand Restaurant (1966)   
10326   148238       A Very Murray Christmas (2015)   
10327   148626                 The Big Short (2015)   
10328   149532  Marco Polo: One Hundred Eyes (2015)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                  

Button(description='Recommend Movies', style=ButtonStyle())

Output()

# COLLABORATIVE BASED SYSTEM

In [2]:
from collab import load_data, cluster_users, get_user_cluster, get_similar_users, get_similar_user_data, calculate_average_rating, get_top_recommendations
import pandas as pd
from sklearn.cluster import KMeans
import ipywidgets as widgets
from IPython.display import display

movies_data = pd.read_csv('movies.csv')
ratings_data = pd.read_csv('ratings.csv')
movie_ratings = pd.merge(movies_data, ratings_data, on='movieId')

# target_user_input = widgets.IntText(description='User ID:')
# num_recommendations_input = widgets.IntText(description='Number of Recommendations (N):')
# min_users_threshold_input = widgets.IntText(description='Minimum Users Threshold (t):')

description = widgets.Label('User ID :')
target_user_input= widgets.IntText(value=0)
target_user_hbox = widgets.HBox([description , widgets.Label('  '), target_user_input])

description = widgets.Label('Minimum Users Threshold (t):')
min_users_threshold_input = widgets.IntText(value=0)
min_users_threshold_hbox = widgets.HBox([description , widgets.Label('  '), min_users_threshold_input])

description = widgets.Label('Number of Recommendations (N):')
num_recommendations_input= widgets.IntText(value=0)
num_recommendations_hbox = widgets.HBox([description , widgets.Label('  '), num_recommendations_input])


recommend_button = widgets.Button(description='Recommend Movies')
target_user_cluster_label = widgets.Label(value='')

recommendations_output = widgets.Output()

n_clusters = 5
movie_ratings = cluster_users(movie_ratings, n_clusters)

def recommend_movies_on_button_click(b):
    target_user = int(target_user_input.value)
    num_recommendations = int(num_recommendations_input.value)
    min_users_threshold = int(min_users_threshold_input.value)

    target_user_cluster = get_user_cluster(target_user, movie_ratings)
    target_user_cluster_label.value = f"Cluster of user {target_user} is {target_user_cluster}"

    similar_users = get_similar_users(target_user_cluster, movie_ratings)
    similar_user_data = get_similar_user_data(similar_users, movie_ratings)
    avg_rating = calculate_average_rating(similar_user_data)

    recommended_movies = get_top_recommendations(target_user, num_recommendations, min_users_threshold, avg_rating, movie_ratings)

    recommendations_output.clear_output()
    with recommendations_output:
        print(f"Top {num_recommendations} Recommended Movies for user {target_user}:")
        display(recommended_movies)

recommend_button.on_click(recommend_movies_on_button_click)
display(target_user_hbox, min_users_threshold_hbox, num_recommendations_hbox, recommend_button, target_user_cluster_label, recommendations_output)



       movieId                                title  \
0            1                     Toy Story (1995)   
1            2                       Jumanji (1995)   
2            3              Grumpier Old Men (1995)   
3            4             Waiting to Exhale (1995)   
4            5   Father of the Bride Part II (1995)   
...        ...                                  ...   
10324   146684        Cosmic Scrat-tastrophe (2015)   
10325   146878           Le Grand Restaurant (1966)   
10326   148238       A Very Murray Christmas (2015)   
10327   148626                 The Big Short (2015)   
10328   149532  Marco Polo: One Hundred Eyes (2015)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                  

C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Program Files\Python310\lib\subprocess.py", line 501, in run
    with Popen(*popenargs, **kwargs) as process:
  File

Button(description='Recommend Movies', style=ButtonStyle())

Label(value='')

Output()

# CONTENT BASED SYSTEM

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display
import difflib
from content import load_movie_data, calculate_similarity, get_movie_recommendations


# Load movie data and calculate similarity matrix
movies_data = load_movie_data('movies.csv')
similarity_matrix = calculate_similarity(movies_data)

# Create IPyWidgets
# movie_name_input = widgets.Text(
#     value='',
#     description='Enter a movie name:',
#     layout = widgets.Layout(padding ='20px')
# )

description = widgets.Label('Enter a movie name:')
movie_name_input = widgets.Text(value='')

movie_hbox = widgets.HBox([description , widgets.Label('  '), movie_name_input])

recommend_button = widgets.Button(
    description='Recommend Movies'
)

recommendation_output = widgets.Output()

# Define the recommend_movies function
def recommend_movies(b):
    with recommendation_output:
        recommendation_output.clear_output()
        movie_name = movie_name_input.value
        movie_recommendations = get_movie_recommendations(movie_name, movies_data, similarity_matrix)

        if movie_recommendations:
            print("Movies suggested for you:")
            for i, (index, score) in enumerate(movie_recommendations):
                if i < 5:
                    recommended_movie = movies_data.iloc[index]['title']
                    print(f'{i + 1}. {recommended_movie}')
        else:
            print("No close match found for the entered movie name.")

# Connect the button to the recommend_movies function
recommend_button.on_click(recommend_movies)

# Display the widgets
display(movie_hbox, recommend_button, recommendation_output)

Button(description='Recommend Movies', style=ButtonStyle())

Output()